In [1]:
# Very early iteration of SQL combination, below is an example

In [2]:
def get_operator(string):
    if '>' in string:
        operator = '>'
    elif '<' in string:
        operator = '<'
    elif '!=' in string:
        operator = '!='
    elif '=' in string:
        operator = '='
    elif ' is not ' in string.lower():
        operator = 'is not'
    elif ' is ' in string.lower():
        operator = 'is'
    return operator

sqls = ["""
SELECT * FROM table
WHERE (numberoflms < 2 OR numberoflms is NULL)
AND lmpath = '1'
AND covid_requested = '1'
AND lm_account = '1'
AND covid_offered_dt is NULL
""",
"""
SELECT * FROM table
WHERE (numberoflms >= 2)
AND lmpath = '1'
AND covid_requested != '1'
AND lm_account = '0'
AND covid_offered_dt is NOT NULL
""",
"""
SELECT * FROM table
WHERE (numberoflms >= 2)
AND lmpath = '1'
AND covid_requested != '0'
AND lm_account = '0'
AND covid_offered_dt is NULL
"""]
union_dict = dict()

for sql in sqls:
    sql_statement = "WHERE" + sql.split('WHERE')[1]
    
    for i in ("AND" + sql_statement.split("WHERE")[1]).split("AND")[1:]:
        i = i.replace('(', '').replace(')','').replace('\n', '')
        if 'or' in i.lower():
            for or_statement in i.replace('(', '').replace(')','').lower().split('or'):
                operator = get_operator(or_statement).strip()
                column = or_statement.lower().split(operator)[0].strip()
                value = or_statement.lower().replace("'", '').split(operator)[-1].strip()
                if column not in union_dict.keys():
                    union_dict[column] = {operator: [value]}
                else:
                    if operator not in union_dict[column].keys():
                        union_dict[column][operator] = [value]
                    else:
                        union_dict[column][operator].append(value)
        else:
            operator = get_operator(i).strip()
            column = i.lower().split(operator)[0].strip()
            value = i.lower().replace("'", '').split(operator)[-1].strip()
            if column not in union_dict.keys():
                union_dict[column] = {operator: [value]}
            else:
                if operator not in union_dict[column].keys():
                    union_dict[column][operator] = [value]
                else:
                    union_dict[column][operator].append(value)

In [3]:
union_dict

{'numberoflms': {'<': ['2'], 'is': ['null'], '>': ['= 2', '= 2']},
 'lmpath': {'=': ['1', '1', '1']},
 'covid_requested': {'=': ['1'], '!=': ['1', '0']},
 'lm_account': {'=': ['1', '0', '0']},
 'covid_offered_dt': {'is': ['null', 'null'], 'is not': ['null']}}

In [4]:
# Make it unique
for col in union_dict.keys():
    for op in union_dict[col].keys():
        union_dict[col][op] = list(set(union_dict[col][op]))
        
new_sql = "WHERE "
num_left = len(union_dict.keys())
for column in union_dict.keys():
    
    multiple_ops = 0
    if len(union_dict[column].keys()) > 1:
        multiple_ops = len(union_dict[column].keys())
    
    open_brackets = False
    
    if multiple_ops > 1:
        new_sql += '('
        open_brackets = True
         
    for operator in union_dict[column].keys():
        value = union_dict[column][operator]
        
        if len(value) == 1:
            value = value[0]
            new_sql += ''.join([column, ' ', operator, ' ', value])
        else:
            value_len = len(value)
            new_sql += '('
            for val in value:
                new_sql += ''.join([column, ' ', operator, ' ', val])
                if value_len > 1:
                    new_sql += ' OR '
                    value_len -= 1
            new_sql += ')'
        if multiple_ops > 1:
            new_sql += ' OR '
            multiple_ops -= 1
    if open_brackets:        
        new_sql += ')'
    
    num_left -= 1
    if num_left != 0:
        new_sql += '\nAND '
new_sql = new_sql.replace("null", 'NULL')

In [6]:
print(sql.split('WHERE')[0]+new_sql)


SELECT * FROM table
WHERE (numberoflms < 2 OR numberoflms is NULL OR numberoflms > = 2)
AND lmpath = 1
AND (covid_requested = 1 OR (covid_requested != 0 OR covid_requested != 1))
AND (lm_account = 0 OR lm_account = 1)
AND (covid_offered_dt is NULL OR covid_offered_dt is not NULL)
